In [ ]:
import os, asyncio
os.environ["OPENAI_API_KEY"] = "Hier API-Key einsetzen"

from ragas.llms import llm_factory
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain.embeddings import OpenAIEmbeddings
from ragas.dataset_schema import SingleTurnSample

# Monkey-Patch (nur für RAGAS 0.2.x)
import ragas.metrics._factual_correctness as _fc
import ragas.metrics._context_recall      as _cr
import ragas.metrics._answer_relevance    as _rr
import ragas.metrics._faithfulness        as _fh
for m in (_fc, _cr, _rr, _fh):
    m.SingleTurnSample = SingleTurnSample

from ragas.metrics import (
    FactualCorrectness as FC,
    ContextRecall     as CR,
    ResponseRelevancy as RR,
    Faithfulness      as FH,
)

# 1) LLM & Embeddings
llm        = llm_factory("gpt-4")
emb_wrapper = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

# 2) Dein Sample
s = SingleTurnSample(
    user_input="Wie warm ist die Jacke X-100?",
    response="Die Jacke X-100 eignet sich für Temperaturen bis -15 °C.",
    reference="Die Jacke X-100 hält Kälte bis −15 °C zuverlässig ab.",
    retrieved_contexts=["Die X-100 Jacke ist wasserdicht, windabweisend und atmungsaktiv."]
)

# 3) Parallel alle vier Scores (jetzt mit Embedding für RR)
fc, cr, rr, fh = asyncio.run(asyncio.gather(
    FC(llm=llm).single_turn_ascore(s),
    CR(llm=llm).single_turn_ascore(s),
    RR(llm=llm, embeddings=emb_wrapper).single_turn_ascore(s),  # ← hier
    FH(llm=llm).single_turn_ascore(s),
))

print(dict(fc=fc, cr=cr, rr=rr, fh=fh))


{'fc': 1.0, 'cr': 0.0, 'rr': 0.9565444040891506, 'fh': 0.0}
